# Preliminaries
## Installation
To be able to run this tutorial, please install the following libraries:

In [ ]:
!pip install bridge-ds
!pip install pycocotools

## Imports

In [ ]:
import tempfile
from pathlib import Path

import holoviews as hv
import panel as pn

from bridge.display.vision import Holoviews
from bridge.providers.vision import Coco2017Detection

hv.extension("bokeh")
pn.extension()

TMP_NOTEBOOK_ROOT = Path(tempfile.mkdtemp()) / "custom_data" / "load_mechanism"

## Load Dataset

In [ ]:
root_dir = TMP_NOTEBOOK_ROOT / "coco"

provider = Coco2017Detection(root_dir, split="val", img_source="stream")
ds = provider.build_dataset(display_engine=Holoviews(bbox_format="xywh"))
ds

# LoadMechanism

In this tutorial we will learn about the **LoadMechanism**, Bridge's way of loading raw data from different sources.

A quick reminder: to access the raw data within each element, we need to use the **SampleAPI**. The column `data` in the **TableAPI** usually (but not always) contains a reference to the data rather than the data itself:


In [ ]:
ds.samples.head(1)

When we want to access data for a given element, we need to call the `element.data` property. In COCO, we have elements for _images_ and for _bboxes_. Because COCO is a **SingularDataset**, every sample has a special element, in this case the image, and annotation elements, in this case the bboxes.

In [ ]:
sample = ds.iget(0)
print("img_data:", sample.data.shape, "\n")

bbox_elements = [ann for ann in sample.elements["bbox"]]
print(*[bb.data for bb in bbox_elements], sep="\n")

Every element holds a **LoadMechanism**, an object responsible for loading data from different sources. In the case of images, `element.data` will perform an HTTP request and load the image in the response. In case for bboxes, which already exist in-memory (i.e. we can see them directly in the `annotations` table, `element.data` will simply load the stored object.

The **LoadMechanism** is defined by two variables:

In [ ]:
img_element = sample.element
print("Image element, loaded over HTTP:")
print("url_or_data:", img_element._load_mechanism.url_or_data)
print("category:", img_element._load_mechanism.category)
print()
print("Bbox elements, loaded from memory:")
print("url_or_data:", bbox_elements[0]._load_mechanism.url_or_data)
print("category:", bbox_elements[0]._load_mechanism.category)

- **url_or_data**, as its name suggests, contains either a url that references the object (url broadly speaking - including local paths, s3 paths, etc.), or contains the actual object, in case we want to store it directly in-memory.
- **category** - accepts a string that is used to determine which logic is used to load the object. Should we load the image using PIL? or a text file using simple `with open()`? this value determines that. To find which categories are supported, use `list_registered_categories`.

## In summary
1. Bridge loads data lazily, only when `element.data` is called
2. The loading mechanism accepts **url_or_data** which defines where to load from (or whatto load), and **category** which defines _how_ to load it.